<a target="_blank" href="https://colab.research.google.com/github/UpstageAI/cookbook/blob/main/cookbooks/upstage/Solar-Full-Stack LLM-101/05_3_OracleDB.ipynb">
<img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

## mmlupro_philosophy TextBook dataset

In [10]:
# set parameters
 
file = open("info/api.txt", "r")
api_key = file.read()
file.close()
file = open("info/datapath.txt", "r")
data_path = file.read()
file.close()
file = open("info/resultspath.txt", "r")
results_path = file.read()
file.close()

In [11]:
from langchain_upstage import UpstageEmbeddings

# 쿼리 전용 임베딩 모델
query_embeddings = UpstageEmbeddings(api_key=api_key, model="solar-embedding-1-large-query")
# 문장 전용 임베딩 모델
passage_embeddings = UpstageEmbeddings(api_key=api_key, model="solar-embedding-1-large-passage")

## 1. build DB (using Wikipida)

### what is the main topic & specific topic of query

In [12]:
# read mmlu_pro.csv file
import pandas as pd
import os

def read_data(data_path):
    data = pd.read_csv(data_path)
    prompts = data['prompts']
    answers = data['answers']
    # returns two lists: prompts and answers
    return prompts, answers

prompts, answers = read_data(os.path.join(data_path, 'mmlupro_test_philosophy.csv'))

In [13]:
testdata = pd.read_csv(data_path+'mmlupro_test_philosophy.csv')
testdata

,prompts,answers
0,QUESTION0) Select the best translation into pr...,A
1,QUESTION1) Construct a complete truth table f...,D
2,QUESTION2) Which of the following proposition...,I
3,QUESTION3) Select the best translation into pr...,A
4,QUESTION4) Select the best translation into pr...,E
...,...,...
494,QUESTION494) Mill claims that in deciding how ...,C
495,QUESTION495) Of the two versions of the princi...,A
496,"QUESTION496) According to Kant, laws of nature...",C
497,QUESTION497) Hare refers to people who endorse...,A


In [14]:
nowtest = pd.DataFrame(columns=['index', 'embed_ques', 'question', 'prompts', 'answers', 'top1', 'top2', 'top3'])

for index, row in testdata.iterrows():
    #if index % 10 != 0 : continue # 일단 실험할 땐 개 단위로 끊어서 가져옴
    q = row.prompts
    a = row.answers
    question = q.partition('(A)')[0]
    question = question.partition(') ')[2]
    q = q.partition(') ')[2]

    try : 
        embedded_query = passage_embeddings.embed_query(question) # 질문만 받아와서 embedding 하기
        nowtest.loc[len(nowtest)] = {'index':index, 'embed_ques' : embedded_query, 'question' : question, 'prompts' : q, 'answers' : a}
    except :
        print(f'pass: {index}')
        continue


pass: 411


In [15]:
nowtest

,index,embed_ques,question,prompts,answers,top1,top2,top3
0,0,"[-0.00475311279296875, 0.00991058349609375, -0...",Select the best translation into predicate log...,Select the best translation into predicate log...,A,NaN,NaN,NaN
1,1,"[0.0004830360412597656, -0.0250091552734375, -...",Construct a complete truth table for the foll...,Construct a complete truth table for the foll...,D,NaN,NaN,NaN
2,2,"[0.004749298095703125, -0.01259613037109375, -...",Which of the following propositions is an imm...,Which of the following propositions is an imm...,I,NaN,NaN,NaN
3,3,"[-0.002498626708984375, -0.007579803466796875,...",Select the best translation into predicate log...,Select the best translation into predicate log...,A,NaN,NaN,NaN
4,4,"[-0.0167694091796875, -0.007282257080078125, -...",Select the best translation into predicate log...,Select the best translation into predicate log...,E,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
493,494,"[-0.0107269287109375, 0.0018281936645507812, -...",Mill claims that in deciding how to act:\n,Mill claims that in deciding how to act:\n(A) ...,C,NaN,NaN,NaN
494,495,"[-0.00873565673828125, -0.0032100677490234375,...",Of the two versions of the principle that Sing...,Of the two versions of the principle that Sing...,A,NaN,NaN,NaN
495,496,"[-0.004878997802734375, -0.0228729248046875, -...","According to Kant, laws of nature are laws acc...","According to Kant, laws of nature are laws acc...",C,NaN,NaN,NaN
496,497,"[-0.017913818359375, -0.0139923095703125, -0.0...",Hare refers to people who endorse ideals witho...,Hare refers to people who endorse ideals witho...,A,NaN,NaN,NaN


In [16]:
nowtest.question[1]

' Construct a complete truth table for the following pairs of propositions. Then, using the truth tables, determine whether the statements are logically equivalent or contradictory. If neither, determine whether they are consistent or inconsistent. Justify your answers.\n(~M ⊃ ~N) ∨ (O ≡ N) and (~M · N) · [(~O ∨ ~N) · (O ∨ N)]\n'

# DB 불러오기

In [17]:
import numpy as np

# .npy 파일 로드 (타입==넘파이)
textbookDB = np.load(data_path+f'embedding/full_philosophy_textbook.npy')
textbookDB = textbookDB.tolist()

textbookDB_embed = np.load(data_path+f'embedding/full_philosophy_textbook_embed.npy')
textbookDB_embed = textbookDB_embed.tolist()

In [18]:
textbookDB_embed[0]

[-0.0251007080078125,
 -0.0134124755859375,
 -0.0230712890625,
 -0.003162384033203125,
 0.0167388916015625,
 0.00783538818359375,
 -0.016845703125,
 -0.0024929046630859375,
 0.005138397216796875,
 -0.043304443359375,
 -0.0277099609375,
 0.0169830322265625,
 0.025360107421875,
 0.0011148452758789062,
 0.01299285888671875,
 0.0002810955047607422,
 0.016998291015625,
 -0.007572174072265625,
 -0.002773284912109375,
 -0.0010118484497070312,
 -0.006744384765625,
 0.000640869140625,
 0.0011186599731445312,
 -3.784894943237305e-05,
 -0.0121612548828125,
 0.0206756591796875,
 -0.0146026611328125,
 0.007244110107421875,
 0.0101165771484375,
 0.01082611083984375,
 0.0196990966796875,
 -0.015838623046875,
 0.006580352783203125,
 -0.01149749755859375,
 -0.0080108642578125,
 -0.01122283935546875,
 0.0118865966796875,
 -0.011199951171875,
 -0.0009655952453613281,
 0.00820159912109375,
 0.00423431396484375,
 -0.007709503173828125,
 0.0045928955078125,
 0.0158233642578125,
 0.00626373291015625,
 0.0366

# 유사도 계산 -> sorting

In [19]:
import numpy as np

for idx, row in nowtest.iterrows() : # 질문 받아오기 
    #if idx == 50 : break

    embed_ques= row.embed_ques

    # 유사도 기준 내림차순 정렬
    sorted_idx = (np.array(embed_ques) @ np.array(textbookDB_embed).T).argsort()[::-1]

    nowtest.at[idx, 'top1'] = textbookDB[sorted_idx[0]]
    nowtest.at[idx, 'top2'] = textbookDB[sorted_idx[1]]
    nowtest.at[idx, 'top3'] = textbookDB[sorted_idx[2]]


In [ ]:
from langchain_core.prompts import PromptTemplate
from langchain_upstage import ChatUpstage


llm = ChatUpstage(api_key = api_key)

prompt_template = PromptTemplate.from_template(
    '''
    
    1. Use the information from the [context] to find the correct answer [answer] to the [question].
    2. Please provide most correct answer.
    
    3. When returning the answer, DO NOT exlain anything. And you must also include the choice number like :
    Answer : (Number) the answer choice
    ---
    
    Question: {question}
    Context: {context1}
    Answer :
    ---
        
    '''

)
zs_chain = prompt_template | llm

responses = []
for idx, row in nowtest.iterrows() :
    #if idx == 100 : break
        
    response = zs_chain.invoke({"question": row.prompts, "context1": row.top1}) # 선지 전까지 받아오기
    responses.append(response.content)
    nowtest.loc[idx, 'predict'] = response.content

responses


In [18]:
nowtest.loc[0]['top1']

'elf-contradictory. So:\n3z God does not exist is logically contingent is true.\n4z If God does not exist is logically contingent is true then Necessarily, God\nexists is false.\n5z Necessarily, God exists is false.T his argument is also valid. Premise 1z is true for reasons exactly analogous to\nthose that favor 1, 1x, and 2y. If it is true that God does not exist is logically\ncontingent then God exists is logically necessary is false. (So then is God has\nmaximal excellence, which entails God exists is logically necessary.) So if God\ndoes not exist is logically contingent is true, then Necessarily, God exists is\nfalse, and that is what premise 4z says. So if premise 2z is true, the Z argument\nis sound and valid, and its conclusion is true. Is 2z true?Summary regarding the Ontological Argument (version 1)\nand the X,Y, and Z argumentsA Each argument is logically valid.\nB In each argument, the first premise is true.\nC In each argument, the fourth premise is true.\nD In each argum

In [ ]:
from langchain_core.prompts import PromptTemplate
from langchain_upstage import ChatUpstage


llm = ChatUpstage(api_key = api_key)

prompt_template = PromptTemplate.from_template(
    '''
    
    Please provide most correct answer. Let's think step by step.
    
    When translating the answer, DO NOT exlain anything. And you must also include the choice number like :
    Answer : (Number) the answer choice
    ---
    
    Question: {question}
    Context: {context1} {context2}
    Answer :
    ---
        
    '''

)
zs_chain2 = prompt_template | llm


responses2 = []
for idx, row in nowtest.iterrows() :
    if idx == 50 : continue

        
    response = zs_chain2.invoke({"question": row.prompts, "context1": row.top1, "context2": row.top2}) # 선지 전까지 받아오기
    responses2.append(response.content)
    nowtest.loc[idx, 'predict'] = response.content

responses2

In [ ]:
nowtest

## Check Accuracy

In [ ]:
# funcion to extract an answer from response

import re

def extract_answer(response):
    """
    extracts the answer from the response using a regular expression.
    expected format: "[ANSWER]: (A) convolutional networks"

    if there are any answers formatted like the format, it returns None.
    """
    pattern = r"\[ANSWER\]:\s*\((A|B|C|D|E)\)"  # Regular expression to capture the answer letter and text
    match = re.search(pattern, response)

    if match:
        return match.group(1) # Extract the letter inside parentheses (e.g., A)
    else:
        return extract_again(response)

def extract_again(response):
    pattern = r"\b[A-J]\b(?!.*\b[A-J]\b)"
    match = re.search(pattern, response)
    if match:
        return match.group(0)
    else:
        return None

In [ ]:
######### 정답 1차 확인 + wrong 뽑아내기 ######

try : del [[wrongdf]]
except : pass
wrongdf = pd.DataFrame(columns=['questionNum', 'answer', '1st_pred'])

# print accuracy
cnt = 0
wrong = []
for idx, (answer, response) in enumerate(zip(answers, responses2)):
    print("-"*10)
    generated_answer = extract_answer(response)
    print(response)
    # check
    if generated_answer:
        print(f"idx: {idx} | generated answer: {generated_answer}, answer: {answer}")
    else:
        print("extraction fail")

    if generated_answer == None:
        wrong.append(idx+1)
        wrongdf.loc[len(wrongdf)] = {"questionNum":idx+1, "answer" : answer, "1st_pred":generated_answer, "2nd_pred":"-"}
        continue
    
    if generated_answer in answer:
        cnt += 1
    else : 
        wrong.append(idx+1)
        wrongdf.loc[len(wrongdf)] = {"questionNum":idx+1, "answer" : answer, "1st_pred":generated_answer, "2nd_pred":None}
        
acc = cnt/100*100
print(f"acc: {acc}%")
print()
print("1st wrong:", wrong)
wrongdf.loc[len(wrongdf)] = {"questionNum":"-", "answer" : "-", "1st_pred":acc}

In [ ]:
wrongdf